In [1]:
# DB 내 작품 정보 가져오기
import pymysql.cursors
import pandas as pd

conn = pymysql.connect(
    host='127.0.0.1',
    port=3306,
    user='root',
    passwd='root',
    db='webnoveldb',
    charset='utf8')

curs = conn.cursor()
sql = "SELECT idnovel, title, serieson FROM novel where serieson is not NULL"
curs.execute(sql)

result = curs.fetchall()
df=pd.DataFrame(result, columns=["id", "novel", "serieson"])
df

,id,novel,serieson
0,2,야한 밤의 레이디오국,https://m.series.naver.com/novel/detail.nhn?pr...
1,5,내 안에 마교있다,https://m.series.naver.com/novel/detail.nhn?pr...
2,6,내 머릿속에 2000년 마법 역사,https://m.series.naver.com/novel/detail.nhn?pr...
3,8,회귀의 전설,https://m.series.naver.com/novel/detail.nhn?pr...
4,9,내가 키운 S급들,https://m.series.naver.com/novel/detail.nhn?pr...
...,...,...,...
16729,32609,나를 키워줘 제발!,https://m.series.naver.com/novel/detail.nhn?pr...
16730,32610,미연시인데 연애를 할 수 없는 건에 대하여,https://m.series.naver.com/novel/detail.nhn?pr...
16731,32611,망나니 소교주로 환생했다,https://m.series.naver.com/novel/detail.nhn?pr...
16732,32612,그 남주와 안전이별하는 방법,https://m.series.naver.com/novel/detail.nhn?pr...


In [2]:
# 네이버 웹소설 삽입
import pandas

df2 = pandas.read_csv('naver_webnovel.csv', encoding='UTF8')
df2

,작품명,링크,연재 플랫폼
0,순수한 동거생활,https://m.novel.naver.com//webnovel/list.nhn?n...,네이버 웹소설
1,매일매일 기대해,https://m.novel.naver.com//webnovel/list.nhn?n...,네이버 웹소설
2,첫날밤만 세 번째,https://m.novel.naver.com//webnovel/list.nhn?n...,네이버 웹소설
3,본능적인 그대,https://m.novel.naver.com//webnovel/list.nhn?n...,네이버 웹소설
4,내 남편과 결혼해줘,https://m.novel.naver.com//webnovel/list.nhn?n...,네이버 웹소설
...,...,...,...
107,"리빌더, 두 번의 후회는 없다",https://m.novel.naver.com//webnovel/list.nhn?n...,네이버 웹소설
108,리턴 투 플레이어,https://m.novel.naver.com//webnovel/list.nhn?n...,네이버 웹소설
109,내가 성공할 확률 100%,https://m.novel.naver.com//webnovel/list.nhn?n...,네이버 웹소설
110,3cm 헌터,https://m.novel.naver.com//webnovel/list.nhn?n...,네이버 웹소설


In [3]:
df2['링크']=df2['링크'].str.replace("/m." ,"/")
df2

,작품명,링크,연재 플랫폼
0,순수한 동거생활,https://novel.naver.com//webnovel/list.nhn?nov...,네이버 웹소설
1,매일매일 기대해,https://novel.naver.com//webnovel/list.nhn?nov...,네이버 웹소설
2,첫날밤만 세 번째,https://novel.naver.com//webnovel/list.nhn?nov...,네이버 웹소설
3,본능적인 그대,https://novel.naver.com//webnovel/list.nhn?nov...,네이버 웹소설
4,내 남편과 결혼해줘,https://novel.naver.com//webnovel/list.nhn?nov...,네이버 웹소설
...,...,...,...
107,"리빌더, 두 번의 후회는 없다",https://novel.naver.com//webnovel/list.nhn?nov...,네이버 웹소설
108,리턴 투 플레이어,https://novel.naver.com//webnovel/list.nhn?nov...,네이버 웹소설
109,내가 성공할 확률 100%,https://novel.naver.com//webnovel/list.nhn?nov...,네이버 웹소설
110,3cm 헌터,https://novel.naver.com//webnovel/list.nhn?nov...,네이버 웹소설


In [17]:
#df_q["id"].iloc[0].to_dict()
type(np.int16(df_q["id"].iloc[0]).item())

int

In [4]:
# 작품 테이블 책 소개 빠진 것 삽입 - serieson(1/n)
import scipy.io
import csv
import pymysql
import datetime
import re
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import numpy as np

total = 0

driver = webdriver.Chrome("C:/project/chromedriver.exe")

conn = pymysql.connect(
    host='127.0.0.1',
    port=3306,
    user='root',
    passwd='root',
    db='webnoveldb',
    charset='utf8')

curs = conn.cursor()
conn.commit()

for i in df2["링크"]:
        driver.get(i)
        source = driver.page_source
        soup=BeautifulSoup(source, 'html.parser')

        link = soup.find('a', class_='lk_series2')
        n_link = link.get('href').replace("https://","https://m.")
        #print(n_link)
        driver.get(n_link)
        #print(driver.current_url.split("&")[0])
        df_q = df[df['serieson'].str.startswith(driver.current_url.split("&")[0])]
        print(df_q["id"].iloc[0])
        sql = """update novel set naverwebnovel = %s where idnovel=%s"""
        curs.execute(sql, (i, np.int16(df_q["id"].iloc[0]).item()))
        total+=1
    
driver.close()

#db의 변화 저장
conn.commit()
conn.close()
print(total)

32597
32595
32360
32588
32337
32538
32583
32586
32469
32561
32418
32551
32422
32427
32380
32598
32401
32379
32594
32423
32587
32582
32527
32349
32377
32424
32433
32536
32398
32576
32366
32372
32436
32539
32406
32375
32339
32358
32434
32405
32500
32579
31775
32470
32580
32525
32534
32575
32439
32504
32506
32367
32501
32599
32499
32415
28532
28834
28886
28478
29212
31649
32590
28548
32341
32471
31395
29048
30968
31705
32475
32548
32474
32577
32397
32355
32608
32532
32581
32535
32408
32550
32407
32498
32503
32413
32399
32526
32564
32428
32374
32373
32593
32592
32409
32547
32414
32560
32528
32400
32589
32600
32529
32410
32364
32420
29085
28044
32549
31950
28518
32357
112


In [10]:
import time
import bs4
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from urllib import parse
import time

driver = webdriver.Chrome("C:/project/chromedriver.exe")

total = 0
total_list =[]
i = 1
pre_title=None

sam_url = "https://novel.naver.com/webnovel/finish.nhn?page={}"
driver.get(sam_url)
soup2 = bs4.BeautifulSoup(driver.page_source, "html.parser")
time.sleep(1)

while True:
    url = sam_url.format(i)
    req = requests.get(url)
    source = req.content
    bs_obj = BeautifulSoup(source, "html.parser")
    #print(bs_obj)

    f_container = bs_obj.find('ul', {'class', 'list_type1 v2 NE=a:lst_end'})
    title = f_container.find('li').find('a').attrs['title']
    if pre_title == title:
        break
    pre_title=title
    con = f_container.findAll('li')

    for li in con:
        f_title = li.find('a').attrs['title']
        f_url = li.find('a').attrs['href']
        print(f_title)
        print("https://m.novel.naver.com/"+f_url)
        total_list.append("https://m.novel.naver.com/"+f_url)
        total = total + 1

    i = i + 1

driver.close()

허니허니 웨딩
https://m.novel.naver.com//webnovel/list.nhn?novelId=466391
반드시 해피엔딩
https://m.novel.naver.com//webnovel/list.nhn?novelId=699566
원하는 건 너 하나
https://m.novel.naver.com//webnovel/list.nhn?novelId=374465
가르쳐 주세요
https://m.novel.naver.com//webnovel/list.nhn?novelId=523286
썸남에서 겟남까지 - 공감백퍼연애백서
https://m.novel.naver.com//webnovel/list.nhn?novelId=552533
구르미 그린 달빛
https://m.novel.naver.com//webnovel/list.nhn?novelId=126772
당신과 나 사이에 없는 것
https://m.novel.naver.com//webnovel/list.nhn?novelId=623312
혼전계약서
https://m.novel.naver.com//webnovel/list.nhn?novelId=799258
너에게로 중독
https://m.novel.naver.com//webnovel/list.nhn?novelId=456722
남자 사람 친구
https://m.novel.naver.com//webnovel/list.nhn?novelId=514809
해시의 신루
https://m.novel.naver.com//webnovel/list.nhn?novelId=398090
아기가 생겼어요
https://m.novel.naver.com//webnovel/list.nhn?novelId=676630
위험한 신혼부부
https://m.novel.naver.com//webnovel/list.nhn?novelId=505096
미치도록 너만을
https://m.novel.naver.com//webnovel/list.nhn?novelId=514807
야한 남자
https://m.novel

판타스틱 남장신부
https://m.novel.naver.com//webnovel/list.nhn?novelId=591700
화비
https://m.novel.naver.com//webnovel/list.nhn?novelId=673014
너라는 명작
https://m.novel.naver.com//webnovel/list.nhn?novelId=545435
유일한 신부
https://m.novel.naver.com//webnovel/list.nhn?novelId=639230
위험천만한 연애
https://m.novel.naver.com//webnovel/list.nhn?novelId=765195
너라는 별
https://m.novel.naver.com//webnovel/list.nhn?novelId=639233
사랑해 악마
https://m.novel.naver.com//webnovel/list.nhn?novelId=554029
악녀의 사정
https://m.novel.naver.com//webnovel/list.nhn?novelId=113901
미스터 군주
https://m.novel.naver.com//webnovel/list.nhn?novelId=583904
운정궁 야사
https://m.novel.naver.com//webnovel/list.nhn?novelId=532198
나에겐 100퍼센트
https://m.novel.naver.com//webnovel/list.nhn?novelId=433830
미쁨이지 아니한가
https://m.novel.naver.com//webnovel/list.nhn?novelId=583900
남주의 첫날밤을 가져버렸다
https://m.novel.naver.com//webnovel/list.nhn?novelId=789670
황제와의 하룻밤
https://m.novel.naver.com//webnovel/list.nhn?novelId=673015
고양이가 있는 서점
https://m.novel.naver.com//webnove

나만의 스킨십 능력자들
https://m.novel.naver.com//webnovel/list.nhn?novelId=523289
앙큼한 거짓말
https://m.novel.naver.com//webnovel/list.nhn?novelId=673011
프로젝트J
https://m.novel.naver.com//webnovel/list.nhn?novelId=7
형의 그녀
https://m.novel.naver.com//webnovel/list.nhn?novelId=423620
홍 의관의 은밀한 비밀
https://m.novel.naver.com//webnovel/list.nhn?novelId=714100
운명의 화살은 제멋대로 날아간다
https://m.novel.naver.com//webnovel/list.nhn?novelId=648616
사랑한다, 변순영!
https://m.novel.naver.com//webnovel/list.nhn?novelId=583906
교룡의 주인
https://m.novel.naver.com//webnovel/list.nhn?novelId=523293
톰과 제린
https://m.novel.naver.com//webnovel/list.nhn?novelId=591705
빨간 거짓말
https://m.novel.naver.com//webnovel/list.nhn?novelId=572377
요운당
https://m.novel.naver.com//webnovel/list.nhn?novelId=374460
세상 끝에서 온 무사
https://m.novel.naver.com//webnovel/list.nhn?novelId=485248
어느 마녀의 회귀록
https://m.novel.naver.com//webnovel/list.nhn?novelId=583902
고결한 상사의 충직한 개
https://m.novel.naver.com//webnovel/list.nhn?novelId=765210
관계의 이해
https://m.novel.naver.

야만의 지구
https://m.novel.naver.com//webnovel/list.nhn?novelId=494451
구해줘
https://m.novel.naver.com//webnovel/list.nhn?novelId=113895
불여귀
https://m.novel.naver.com//webnovel/list.nhn?novelId=140735
비밀의 공범
https://m.novel.naver.com//webnovel/list.nhn?novelId=782126
합의해주세요
https://m.novel.naver.com//webnovel/list.nhn?novelId=834844
광야의 야수들
https://m.novel.naver.com//webnovel/list.nhn?novelId=466392
마왕의 올리브
https://m.novel.naver.com//webnovel/list.nhn?novelId=772990
블러셔와 컨실러
https://m.novel.naver.com//webnovel/list.nhn?novelId=63069
위험한 펫
https://m.novel.naver.com//webnovel/list.nhn?novelId=598681
입 맞추고 시작
https://m.novel.naver.com//webnovel/list.nhn?novelId=775596
젤리 블랙 라떼
https://m.novel.naver.com//webnovel/list.nhn?novelId=789669
오늘 밤만 재워줘
https://m.novel.naver.com//webnovel/list.nhn?novelId=746493
천마류혼기
https://m.novel.naver.com//webnovel/list.nhn?novelId=333312
화공, 해란
https://m.novel.naver.com//webnovel/list.nhn?novelId=746495
키스의 품격
https://m.novel.naver.com//webnovel/list.nhn?novelId=

마인드 헌터
https://m.novel.naver.com//webnovel/list.nhn?novelId=231622
그냥 장난이었어
https://m.novel.naver.com//webnovel/list.nhn?novelId=153309
용사, 그만두겠습니다
https://m.novel.naver.com//webnovel/list.nhn?novelId=753191
개미들
https://m.novel.naver.com//webnovel/list.nhn?novelId=50518
마이퍼스트레이디
https://m.novel.naver.com//webnovel/list.nhn?novelId=188292
맥시멈 에이지
https://m.novel.naver.com//webnovel/list.nhn?novelId=664251
진혼
https://m.novel.naver.com//webnovel/list.nhn?novelId=664252
출출한 남자
https://m.novel.naver.com//webnovel/list.nhn?novelId=188295
아이돌 납치사건
https://m.novel.naver.com//webnovel/list.nhn?novelId=291830
친절한 종화씨의 민간인 사찰 보고서
https://m.novel.naver.com//webnovel/list.nhn?novelId=98993
엔딩 크레딧
https://m.novel.naver.com//webnovel/list.nhn?novelId=648615
황태자 납치 사건
https://m.novel.naver.com//webnovel/list.nhn?novelId=809018
빈틈없이 고요하게
https://m.novel.naver.com//webnovel/list.nhn?novelId=614747
오뉘탑: 퇴마사건일지
https://m.novel.naver.com//webnovel/list.nhn?novelId=782131
킹스 앤 데빌
https://m.novel.naver.com//

In [12]:
total_list

['https://m.novel.naver.com//webnovel/list.nhn?novelId=466391',
 'https://m.novel.naver.com//webnovel/list.nhn?novelId=699566',
 'https://m.novel.naver.com//webnovel/list.nhn?novelId=374465',
 'https://m.novel.naver.com//webnovel/list.nhn?novelId=523286',
 'https://m.novel.naver.com//webnovel/list.nhn?novelId=552533',
 'https://m.novel.naver.com//webnovel/list.nhn?novelId=126772',
 'https://m.novel.naver.com//webnovel/list.nhn?novelId=623312',
 'https://m.novel.naver.com//webnovel/list.nhn?novelId=799258',
 'https://m.novel.naver.com//webnovel/list.nhn?novelId=456722',
 'https://m.novel.naver.com//webnovel/list.nhn?novelId=514809',
 'https://m.novel.naver.com//webnovel/list.nhn?novelId=398090',
 'https://m.novel.naver.com//webnovel/list.nhn?novelId=676630',
 'https://m.novel.naver.com//webnovel/list.nhn?novelId=505096',
 'https://m.novel.naver.com//webnovel/list.nhn?novelId=514807',
 'https://m.novel.naver.com//webnovel/list.nhn?novelId=374464',
 'https://m.novel.naver.com//webnovel/li

In [20]:
# 네이버 웹소설 완결작 추가
import scipy.io
import csv
import pymysql
import datetime
import re
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import numpy as np

total = 0

driver = webdriver.Chrome("C:/project/chromedriver.exe")

conn = pymysql.connect(
    host='127.0.0.1',
    port=3306,
    user='root',
    passwd='root',
    db='webnoveldb',
    charset='utf8')

curs = conn.cursor()
conn.commit()

for i in total_list:
    try:
        driver.get(i.replace("https://m.","https://"))
        source = driver.page_source
        soup=BeautifulSoup(source, 'html.parser')

        link = soup.find('a', class_='lk_series2')
        n_link = link.get('href').replace("https://","https://m.")
        #print(n_link)
        driver.get(n_link)
        #print(driver.current_url.split("&")[0])
        df_q = df[df['serieson'].str.startswith(driver.current_url.split("&")[0])]
        print(df_q["id"].iloc[0])
        
        sql = """update novel set naverwebnovel = %s where idnovel=%s"""
        curs.execute(sql, (i, np.int16(df_q["id"].iloc[0]).item()))
        total+=1
    except:
        print(i)
        continue
    
driver.close()
print(total)

#db의 변화 저장
conn.commit()
conn.close()

25676
27557
25693
25965
https://m.novel.naver.com//webnovel/list.nhn?novelId=552533
26531
25794
29197
25774
25920
25660
27442
25743
25949
25690
27488
25744
25773
25964
25947
25952
26600
27994
https://m.novel.naver.com//webnovel/list.nhn?novelId=4
26559
26576
https://m.novel.naver.com//webnovel/list.nhn?novelId=188284
28511
26578
25688
27432
25908
25705
26598
25619
25800
27301
26543
25681
25939
28544
27299
32569
25970
27536
27440
28369
28405
32562
28382
26539
25772
25942
25944
https://m.novel.naver.com//webnovel/list.nhn?novelId=775593
28957
25948
28415
28380
25726
26519
26577
29009
27359
25731
25911
25896
25966
28935
25742
28643
https://m.novel.naver.com//webnovel/list.nhn?novelId=307727
25977
29169
25662
32361
28695
26568
28524
25703
32368
https://m.novel.naver.com//webnovel/list.nhn?novelId=84285
25937
25935
27304
https://m.novel.naver.com//webnovel/list.nhn?novelId=398097
27531
25799
25963
25953
29165
25692
28525
25885
25725
25940
25898
19642
28441
27379
25727
https://m.novel.naver.

In [ ]:
# https://m.series.naver.com/novel/detail.nhn?productNo=2903039 책 링크
# https://m.series.naver.com/novel/moreDetail.nhn?productNo=2903039 책소개 링크

# 네이버 로그인
driver.get('https://nid.naver.com/nidlogin.login')
naver_id = "shung2rhea"
naver_pw = "rhea155204"
driver.execute_script("document.getElementsByName('id')[0].value=\'" + naver_id + "\'")
# time.sleep(1)
driver.execute_script("document.getElementsByName('pw')[0].value=\'" + naver_pw + "\'")
# time.sleep(1)
driver.find_element_by_xpath('//*[@id="frmNIDLogin"]/fieldset/input').click()

time.sleep(0.5) ## 0.5초

In [31]:
# 작품 테이블 책 소개 빠진 것 삽입 - serieson(1/n)
import scipy.io
import csv
import pymysql
import datetime
import re
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time

driver = webdriver.Chrome("C:/project/chromedriver.exe")

# https://m.series.naver.com/novel/detail.nhn?productNo=2903039 책 링크
# https://m.series.naver.com/novel/moreDetail.nhn?productNo=2903039 책소개 링크

# 네이버 로그인
driver.get('https://nid.naver.com/nidlogin.login')
naver_id = "shung2rhea"
naver_pw = "rhea155204"
driver.execute_script("document.getElementsByName('id')[0].value=\'" + naver_id + "\'")
# time.sleep(1)
driver.execute_script("document.getElementsByName('pw')[0].value=\'" + naver_pw + "\'")
# time.sleep(1)
driver.find_element_by_xpath('//*[@id="frmNIDLogin"]/fieldset/input').click()

time.sleep(0.5) ## 0.5초

conn = pymysql.connect(
    host='127.0.0.1',
    port=3306,
    user='root',
    passwd='root',
    db='webnoveldb',
    charset='utf8')

curs = conn.cursor()
conn.commit()


for i in result:
    print(i)
    id_num = i[0]
    introduction = ""
    intro_url = "https://m.series.naver.com/novel/moreDetail.nhn?"+i[2].split("?")[1]
    
    driver.get(intro_url)
    source = driver.page_source
    soup=BeautifulSoup(source, 'html.parser')
    
    for i in soup.select('#ct > div > div.end_txt2'):
        introduction+=i.text
    print(introduction+"\n")
    
    sql = """update novel set intro = %s where idnovel=%s"""
    curs.execute(sql, (introduction, id_num))
    
driver.close()

#db의 변화 저장
conn.commit()
conn.close()

(25339, '당신이 원하는 대로', 'https://m.series.naver.com/novel/detail.nhn?productNo=2857424')
잔혹한 마피아 세계를 장악한 섬뜩한 야수와 
얌전한 꼭두각시 인형에서 한 남자에 대한 사랑으로 피어나는 여인의 핏빛 로맨스.

유일하게 자유를 안겨준 주환의 청혼을 수락한 가녀리고 순수한 여인, 은새. 
결혼 발표 날 순백의 그녀 앞에 시리도록 푸른 눈을 가진 아름답고도 치명적인 남자, 루이가 다가온다.

“재미있는 걸 발견했거든.”

태어나 처음으로 자신의 시선을 빼앗아간 은새를 갖기 위해 루이는 그녀의 결혼 발표 현장에서 잔혹한 납치극을 벌인다. 

약혼자의 차를 으스러뜨린 트럭, 주환의 이름을 비명 섞인 목소리로 부르는 은새의 앞에 그가 나타난다.

빛나는 금발, 어둠 속에서도 빛나는 사파이어같이 푸른 두 눈동자. 그 기억을 마지막으로 은새는 정신을 잃고 마는데….

(25342, '아르디안의 연인', 'https://m.series.naver.com/novel/detail.nhn?productNo=2903129')
지도에도 없는 나라, 태양의 제국 헤르디스...
일 년에 딱 하루 일 분간만 그곳으로 가는 문이 열린다. 
헤르디스  제국의 젊은 국왕 아르디안과 고단한 이땅의 공시생 채라가 운명적으로 만나 우여곡절끝에 도달하는 사랑...
헤르디스로 들어간 채라가 겪는 모험과 로맨스...
아르디안의 연인.

(25344, '우아한 짐승과의 결혼 외전', 'https://m.series.naver.com/novel/detail.nhn?productNo=2913640')
필요에 의해 결혼으로 맺어졌던 강준과 윤희. 
<우아한 짐승과의 결혼>의 뒷 이야기가 궁금하셨던 분들을 위한 서비스 외전! 
건전한 본편의 아쉬움을 달래 줄 아찔한 19신부터 
감초 노릇을 톡톡히 했던 조카들과의 이야기까지 
본편만큼 알찬 외전, 놓치지 말고 챙겨 가세요~!

(25345, '원나잇 온리', 'https://m.series.n

바이올린을 하는 연상의 다정하고 상냥한 남자친구, 수인!
금수저에 노는 걸 좋아하는 야생마 같은 애인, 주영!
두 커플의 삐그덕거리는 친목 이야기에 약간의 로맨스릴러.

(25391, '러닝', 'https://m.series.naver.com/novel/detail.nhn?productNo=2586473')
조용히 묻어 둔 감정일 뿐이었다.
김규원. 손을 내밀어 만져 볼 수 없는 남자.
하지만 별일이다 싶은 밤.
술기운을 빌려 내밀어 본 손.
그리고 그렇게 일이 생겨버렸다.

“박서혜 마음까지 원하면 안 되냐고. 둘 다 가질 수 있다면 갖는 것도 나쁘지 않잖아.”
규원의 삐딱한 미소에 서혜의 마음이 무너져 내린다.
“그렇게까지 말 할 필요 없잖아요.”
자신의 마음은 내어주지 않으면서 그녀의 마음까지 달라고 하는 그가 잔인하게 느껴졌다.
“알았어. 당신이랑 자는 거 좋아하지만 그거 못해서 죽는 것도 아니니까.”
아무렇지 않은 척 덤덤한 말로 상처를 주고 상처를 입는다.
“마음 바뀌면 말해. 당신은 어떨지 몰라도 아직 나는 당신 몸에 질리지 않았으니까.”

Z에서부터 시작하는 연애.
하지만 난 항상 당신뿐이었어요.

「당신에게, 러닝」


#사내연애, #원나잇, #짝사랑, #고수위, #순정남

(25401, '네버 세이 굿바이', 'https://m.series.naver.com/novel/detail.nhn?productNo=2501415')
“너만 태어나지 않았어도 됐어."
어둠이 내리깔린 새벽, 익숙한 목소리가 중얼거렸다. 
그 음성은 한 자락 연기처럼 소녀의 숨통을 짓눌렀다. 
그 날 이후였다. 천천히, 아주 천천히 저주에 잠식되고 만 것은.

해나는 그렇게 스스로를 저주 속에 가둬버렸다.
 
“너만 있어주면 돼."
어릴 적부터 소년에겐 소녀밖에 없었다. 
하지만 마음을 주면 줄수록 소녀는 뒷걸음친다. 
그런데도 그놈의 마음이란 게 자꾸만 깊어지기만 한다. 
마음 둘 곳 없는 그녀를 넓은 가슴에 품어주고 싶다. 
 
그래서 무진은 어른이 되었다. 
 
사

"떴다, 그녀!!! 그 이름도 유명한 도운비. 그녀의 레이더에 걸린 남자, 최원찬. 그리고 또 한 명의 남자, 나진건. 좌충우돌, 캠퍼스 별종들의 특별한 something! 스물한 살의 남자. 원숙미는 없을지라도 그 설익은 섹시미가 운비의 시선을 더 잡아끌었다. 

‘너 나한테 찍혔어, 인마.’ - 도깨비 

한마디로 그녀는 건강 미인이었다. 그리고 무척…… 섹시했다. 

‘캠퍼스의 전설, 도깨비의 단독 인터뷰를 따내다.’ - 최원찬 

이조영의 로맨스 장편 소설 『캠퍼스에 도깨비 뜨다』."

(25514, '뽀앵뜨 뽀앵뜨', 'https://m.series.naver.com/novel/detail.nhn?productNo=2098346')
"“당신은 내 거야.” 
그녀의 고백이 그를 황홀하게 만들었다. 선현은 손바닥을 들어 매끄러운 그의 가슴을 쓸었다. 그녀의 손이 스치는 곳마다 은석의 피부가 뜨겁게 달아올랐다. 
“누구에게도 안 줘.” 
그녀의 손가락이 그의 유두를 희롱했다. 점차로 딱딱해지는 감촉을 즐기면서 깜빡이는 선현의 긴 속눈썹이 은석의 살갗을 스쳤다. 그는 그녀를 와락 껴안았다. 

속죄하기 위해 사랑하기로 한 여자. 
그러나 언제부턴가 그 속죄를 즐기고 있음을 깨닫는다. 

신이여, 용서하세요. 단 한 번만. 

조효은의 로맨스 장편 소설 『뽀앵뜨 뽀앵뜨』"

(25516, '레디 액션!', 'https://m.series.naver.com/novel/detail.nhn?productNo=2094602')
"영화에 대한 열정으로 똘똘 뭉친 감독, 서진. 
직접 쓴 <이글스 레이스>라는 시나리오의 제작비를 벌기 위해 에로 영화의 감독으로 나섰다. 

그리고 그녀가 점찍은 신인 영화배우, 준혁. 
제임스 딘처럼 까칠한 반항아 이미지에 선한 눈매를 가진 그는 여심을 단숨에 사로잡을 정도로 매력적이다. 

하지만 기획사의 등살에 억지로 에로 영화의 오디션에 참석한 준혁은 서진의 영화를 삼류로 매도하고, 그렇게 그들은 첫 만남부터 악연을 시작한다. 

까

에피루스 베스트 로맨스 연재 소설!

같은 날, 같은 병원, 같은 분만실에서 태어난 것도 모자라
너무도 친한 두 어머니…….
소꿉친구인데…….
우정이상 아닌데…….
어느새 사랑할 나이가 되었다 싶을 때
연인이 되기도 전에 부부가 된 두 사람, 승준과 하율.
연인은 달콤할지 모르겠지만 부부라서 뜨거운 두 사람
과연 사랑일 것인가…….

(25569, '이층집', 'https://m.series.naver.com/novel/detail.nhn?productNo=1807653')
이층집에서 우연히 보게된 한 아름다운 소년을 가슴에 품고 살던 준희는 십년후 그 이층집에서 남장을 하며 과외를 하던중 첫사랑의 조카인 현기와 사랑에 빠진다.

얼마후 현기는 심장병으로 죽게되고 삶의 의미를 잃은 주인공 준희는 첫사랑이라고 생각했던 지선우의 수하로 들어가 일을 하면서 칼이라는 고객을 만나고 그가 현기일지 모른다는 생각에

끊임없이 혼란스러워 하던중 지선우의 정체가 서서히 드러나게 되는데...

(25570, '사장님은 바람둥이', 'https://m.series.naver.com/novel/detail.nhn?productNo=1895087')
사장님은 바람둥이에 조폭 출신?
바람둥이 사장님 선도에 나서는 반소녀, 바람둥이의 매력에 빠지다!
달콤 살벌한 그들의 로맨스... 사장님은 바람둥이!

(25585, '그녀는 내 아내다', 'https://m.series.naver.com/novel/detail.nhn?productNo=1764316')
선 자리에서 다시 만난 동생을 좋아한다며 쫒아 다녔던 소녀 설희와 결혼을 했다.
아내가 되었으니 행복하리라 생각했다.
하지만,
아내의 닫힌 마음은 열릴 줄 모르고 끝내 그를 떠나려 하는데…….
결혼만 하면 내 여자라 생각했었고
결혼했으니 내 아내라 생각했었었던 태준은…….
아내의 기억을 돌려놓고 싶은 남편

(25587, '소꿉사랑', 'https://m.series.naver.com/novel/detail.nhn?productNo=1

가족 같이 지냈던 친구가 남자로 보이기 시작했다!
나인의 재원 꼬시기 프로젝트!! “재원아, 남자를 가르쳐줘.”

“아~ 너무 좋아.”
오늘도 어김없이 들려오는 여자의 신음소리에 나인은 신경질 적으로 방문은 쾅! 닫고 들어가 버린다. 남의 집에 아무렇지도 않게 여자를 불러들이는 낯 두꺼운(?) 놈은 볼일을 다 마쳤는지 여자와 함께 방에서 나와 유유히 사라진다. 나인은 방으로 들어가 남녀가 뒹군 흔적들을 치운다. 그런데 왜 이렇게 서러운 걸까. 왜 자꾸 이재원, 그 자식의 얼굴이 떠오르는 걸까.
같은 날 부모님을 잃게 된 나인과 재원은 서로를 의지하며 지낸 가족 같은 사이지만, 언젠가부터 나인의 마음에서 재원은 점점 커진다. 하지만 여러 여자를 만나고 다니는 재원에게 나인은 도저히 고백할 용기를 내지 못하고, 친구인 정아에게 이 사실을 털어놓는다. 정아는 곰곰이 생각해보다가 한 가지 묘안을 나인에게 일러준다.
처음엔 정아의 말을 듣고 당황했지만, 일단 나인은 정아가 하라는 대로 재원에게 말한다.
“넌 앞으로 나한테 남자를 가르쳐 주면 돼.”
“뭐?”
“남자를 가르쳐 달라고.”

남자를 가르쳐 달라고?
그게 어떻게 하면 알려줄 수 있는 건데?

재원은 어안이 벙벙한 표정으로 나인을 바라보았다. 이미 여기까지 저지른 거 멈추기에는 늦었다고 판단한 나인은 바로 재원에게 들이댄다.
“키스… 가르쳐줘!”
뭘 가르쳐줘? 키스를?? 당황하는 재원에게 들어가는 나인의 필살 카운터펀치!!
“키스, 섹스… 모두 다!!”
여자라면 질리도록 겪어본 재원이었지만, 이런 경우는 처음이었다. 하지만 재원은 마음에 평정심을 찾고 피식, 웃더니 나인의 입술로 자신의 입술을 가져간다.
“이게 키스야, 안나인.”
나인은 처음 해보는 키스에 정신을 못 차리고 숨을 고르던 나인은 이러다 질식해서 죽기라도 하는 건 아닌지 덜컥, 겁이 난다.
남자를 너무 모르는 나인과, 여자를 너무 잘 아는 재원. 이 아슬아슬한 수업의 끝에서 나인이 받을 성적표는 과연 무엇일까?

남자를 가르쳐줘요 / 김지혜 / 로맨

In [2]:
# 작품 테이블 나이 정보 미제공 삽입 수정 - serieson(1/n)(2/n)(3/n)(4/n)
import scipy.io
import csv
import pymysql
import datetime
import re
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time

driver = webdriver.Chrome("C:/project/chromedriver.exe")

# https://m.series.naver.com/novel/detail.nhn?productNo=2903039 책 링크
# https://m.series.naver.com/novel/moreDetail.nhn?productNo=2903039 책소개 링크

# 네이버 로그인
driver.get('https://nid.naver.com/nidlogin.login')
naver_id = "shung2rhea"
naver_pw = "rhea155204"
driver.execute_script("document.getElementsByName('id')[0].value=\'" + naver_id + "\'")
# time.sleep(1)
driver.execute_script("document.getElementsByName('pw')[0].value=\'" + naver_pw + "\'")
# time.sleep(1)
driver.find_element_by_xpath('//*[@id="frmNIDLogin"]/fieldset/input').click()

time.sleep(0.5) ## 0.5초

conn = pymysql.connect(
    host='127.0.0.1',
    port=3306,
    user='root',
    passwd='root',
    db='webnoveldb',
    charset='utf8')

curs = conn.cursor()
conn.commit()

for i in result:
    print(i)
    id_num = i[0]
    age = ""
    intro_url = "https://m.series.naver.com/novel/moreDetail.nhn?"+i[3].split("?")[1]
    
    driver.get(intro_url)
    source = driver.page_source
    soup=BeautifulSoup(source, 'html.parser')

    age = soup.find('dl',class_='txt_info txt_info_v5').find_all('dd')[3].text
    if '12세' in age:
        age='12세연령가'
    elif '15세' in age:
        age='15세연령가'
    elif '전체' in age:
        age = '전체연령가'
    elif '청소년' in age:
        age='19세연령가'
    else:
        age='미제공'
    print(age)
    
    sql = """update novel set age = %s where idnovel=%s"""
    curs.execute(sql, (age, id_num))
    
driver.close()

#db의 변화 저장
conn.commit()
conn.close()

(32337, '내 남편과 결혼해줘', '미제공', 'https://m.series.naver.com/novel/detail.nhn?productNo=4812845')
전체연령가
(32338, '마도조사', '미제공', 'https://m.series.naver.com/novel/detail.nhn?productNo=4534735')
15세연령가
(32339, '부적절한 유혹', '미제공', 'https://m.series.naver.com/novel/detail.nhn?productNo=5144073')
전체연령가
(32340, '미래를 보는 투자자', '미제공', 'https://m.series.naver.com/novel/detail.nhn?productNo=3091861')
전체연령가
(32341, '천하제일역졸', '미제공', 'https://m.series.naver.com/novel/detail.nhn?productNo=3847113')
전체연령가
(32342, '중증외상센터 : 골든 아워', '미제공', 'https://m.series.naver.com/novel/detail.nhn?productNo=4033538')
전체연령가
(32343, '처음입니다', '미제공', 'https://m.series.naver.com/novel/detail.nhn?productNo=5050619')
15세연령가
(32344, '개굴맨의 연애백서', '미제공', 'https://m.series.naver.com/novel/detail.nhn?productNo=5381175')
12세연령가
(32345, '악법도 사랑이다', '미제공', 'https://m.series.naver.com/novel/detail.nhn?productNo=5247448')
15세연령가
(32346, '출구는 없습니다', '미제공', 'https://m.series.naver.com/novel/detail.nhn?productNo=5024350')
15세연령가
(32347, '언터처블 

12세연령가
(32420, '갬블링 1945', '미제공', 'https://m.series.naver.com/novel/detail.nhn?productNo=3105317')
전체연령가
(32421, '튜토리얼 탑의 고인물', '미제공', 'https://m.series.naver.com/novel/detail.nhn?productNo=4230091')
전체연령가
(32422, '당신만 모르는 사이', '미제공', 'https://m.series.naver.com/novel/detail.nhn?productNo=5331844')
전체연령가
(32423, '너무 잘 보이는 그녀', '미제공', 'https://m.series.naver.com/novel/detail.nhn?productNo=4653836')
전체연령가
(32424, '말 많은 연애', '미제공', 'https://m.series.naver.com/novel/detail.nhn?productNo=4812843')
전체연령가
(32425, '폐하의 어린 신랑님', '미제공', 'https://m.series.naver.com/novel/detail.nhn?productNo=5408989')
15세연령가
(32426, '그 황후를 믿지 마세요', '미제공', 'https://m.series.naver.com/novel/detail.nhn?productNo=5365518')
15세연령가
(32427, '나랑 해요', '미제공', 'https://m.series.naver.com/novel/detail.nhn?productNo=4983640')
전체연령가
(32428, '황제 부부가 자꾸 윙크합니다', '미제공', 'https://m.series.naver.com/novel/detail.nhn?productNo=5331950')
전체연령가
(32429, '첫날밤엔 리허설이 없다', '미제공', 'https://m.series.naver.com/novel/detail.nhn?productNo=549950

전체연령가
(32499, '언더커버 아일랜드', '미제공', 'https://m.series.naver.com/novel/detail.nhn?productNo=4983643')
전체연령가
(32500, '완벽한 파트너', '미제공', 'https://m.series.naver.com/novel/detail.nhn?productNo=4983644')
전체연령가
(32501, '짝사랑을 완결합니다', '미제공', 'https://m.series.naver.com/novel/detail.nhn?productNo=4812832')
전체연령가
(32502, '입술이 너무해', '미제공', 'https://m.series.naver.com/novel/detail.nhn?productNo=3200031')
전체연령가
(32503, '후궁의 초대', '미제공', 'https://m.series.naver.com/novel/detail.nhn?productNo=4402559')
전체연령가
(32504, '어느 날 남친을 분실했다', '미제공', 'https://m.series.naver.com/novel/detail.nhn?productNo=5068203')
전체연령가
(32505, '두 번 사는 프로듀서', '미제공', 'https://m.series.naver.com/novel/detail.nhn?productNo=4902776')
전체연령가
(32506, '그 밤을 걷다 너를 주웠다', '미제공', 'https://m.series.naver.com/novel/detail.nhn?productNo=5144066')
전체연령가
(32507, '여우신은 신부를 사랑한다', '미제공', 'https://m.series.naver.com/novel/detail.nhn?productNo=5499910')
15세연령가
(32508, '토끼 신부가 왔습니다', '미제공', 'https://m.series.naver.com/novel/detail.nhn?productNo=5418948'

전체연령가
(32581, '고수, 후궁으로 깨어나다', '미제공', 'https://m.series.naver.com/novel/detail.nhn?productNo=5144065')
전체연령가
(32582, '치명적인 내 여자', '미제공', 'https://m.series.naver.com/novel/detail.nhn?productNo=4812851')
전체연령가
(32583, '오빠집이 비어서', '미제공', 'https://m.series.naver.com/novel/detail.nhn?productNo=4983641')
전체연령가
(32584, '간택주의보', '미제공', 'https://m.series.naver.com/novel/detail.nhn?productNo=5529879')
전체연령가
(32585, '집착 말고, 이혼해 주세요!', '미제공', 'https://m.series.naver.com/novel/detail.nhn?productNo=5431988')
전체연령가
(32586, '팔려 온 신부', '미제공', 'https://m.series.naver.com/novel/detail.nhn?productNo=4649889')
전체연령가
(32587, '공주, 폭군을 유혹하다', '미제공', 'https://m.series.naver.com/novel/detail.nhn?productNo=4355064')
전체연령가
(32588, '본능적인 그대', '미제공', 'https://m.series.naver.com/novel/detail.nhn?productNo=4402555')
전체연령가
(32589, '후회는 남기지 않는다', '미제공', 'https://m.series.naver.com/novel/detail.nhn?productNo=4650005')
전체연령가
(32590, '천하제일 대사형', '미제공', 'https://m.series.naver.com/novel/detail.nhn?productNo=3362193')
전체연령가

In [19]:
conn.close()